<a href="https://colab.research.google.com/github/DDimovv/OPJ/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets evaluate bert_score sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from evaluate import load
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from seq2seq import create_transformers_train_data, train_transformer, decode_with_transformer

In [ ]:
data = pd.read_csv('test_en_parallel.txt', sep='\t', header=None)
data = data[[0, 1]]
data.columns = ['NEGATIVE', 'POSITIVE']
data = data[:999]
sentences_ne = data['NEGATIVE'].values.tolist()
sentences_pos = data['POSITIVE'].values.tolist()

In [ ]:
model_name = 't5-small'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
sentences_ne[0]

'Style 1'

In [ ]:
sentences_ne = sentences_ne[1:]
sentences_pos = sentences_pos[1:]

In [ ]:
train_set = create_transformers_train_data(sentences_ne, sentences_pos, tokenizer)
train_set

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 998
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [ ]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [ ]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 2.9557
Epoch 2/5, Loss: 2.3372
Epoch 3/5, Loss: 2.1531
Epoch 4/5, Loss: 1.9854
Epoch 5/5, Loss: 1.8564


In [ ]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in sentences_ne
]


In [ ]:
predicted_sentences

['ever since joes has changed hands',
 'there is definitely enough room in that part of',
 'so basically tasted watered down.',
 "she said she'd be back and disappeared",
 "i can't believe how inconsider",
 'just left and took it off the bill.',
 "it is great, but it isn'",
 'definitely disappointed that i could use my birthday',
 'new owner, i heard - but',
 'it probably sucks too!',
 'we sit down and get some really good service',
 'the charge included miso soup and a',
 "there was no i'm sorry or",
 'we were ordering dinner at the table.',
 'the cash register area was great and everyone was',
 'there are some good chips, but their salsa',
 'the wine was very good and the food was',
 'staffed by teenagers who understand customer service',
 'the burgers were cooked to the point',
 'blue cheese dressing was the best by any means',
 'my pad thai tasted like',
 "she said 'yes, sorry'",
 'the store is clean and the management needs to',
 'there was only meat and bread.',
 'when i complain

In [ ]:
reference_sentences = sentences_pos
reference_sentences

["Ever since joes has changed hands it's gotten better and better.",
 'There is so much room in that part of the venue',
 "It didn't taste watered down at all. ",
 "She said she'd be back, and didn't disappear at all. ",
 'This pharmacy is really considerate.',
 'just left and put it on the bill.',
 "it isn't perfect, but it is very good.",
 'definitely not disappointed that i could use my birthday gift!',
 "great new owner, i heard - but i don't know the details.",
 "but it probably doesn't suck too!",
 'the service was quick and responsive',
 'the price included miso soup and a great salad',
 'they asked how everything was',
 'they said we could sit at the table with no hesitation',
 'the store front was well attended',
 'these chips are okay but their salsa is really tasty',
 'the wine was above average and the food was even better',
 'staffed primarily by teenagers who understand customer service',
 'the burgers were cooked perfectly and the meat was juicy',
 'blue cheese dressing 

In [ ]:
bleu = load('bleu')

In [ ]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.26308657526261825,
 'precisions': [0.6080493619408217,
  0.42540355454100764,
  0.3127555988315482,
  0.23641632455928518],
 'brevity_penalty': 0.7074456183810502,
 'length_ratio': 0.7428898843629544,
 'translation_length': 7131,
 'reference_length': 9599}

In [ ]:
bertscore = load('bertscore')

In [ ]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

{'precision': [0.8664426803588867,
  0.7905800342559814,
  0.7682116031646729,
  0.7883107662200928,
  0.5715967416763306,
  0.8572221994400024,
  0.6497834920883179,
  0.8551269173622131,
  0.8138712644577026,
  0.8690667152404785,
  0.6494177579879761,
  0.8722113370895386,
  0.4686126708984375,
  0.7251790761947632,
  0.6637405157089233,
  0.7458484768867493,
  0.837470293045044,
  0.9325549602508545,
  0.8490806818008423,
  0.6827027797698975,
  0.8216825127601624,
  0.6641874313354492,
  0.6030565500259399,
  0.804516077041626,
  0.6872538328170776,
  0.9044262170791626,
  0.7351149320602417,
  0.7684889435768127,
  0.7030043601989746,
  0.8031405806541443,
  0.9090107679367065,
  0.9906212687492371,
  0.7377029061317444,
  0.9093289971351624,
  0.7158664464950562,
  0.5848265886306763,
  0.7206916213035583,
  0.6860220432281494,
  0.6425478458404541,
  0.5533080101013184,
  0.781125545501709,
  0.7633999586105347,
  0.657865047454834,
  0.8891733884811401,
  0.7909694910049438,
 

In [ ]:
model2 = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
optimizer2 = AdamW(model2.parameters(), lr=0.005)

In [ ]:
train_transformer(model2, train_loader, optimizer2, 5)

Epoch 1/5, Loss: 3.0742
Epoch 2/5, Loss: 2.2984
Epoch 3/5, Loss: 1.9088
Epoch 4/5, Loss: 1.6020
Epoch 5/5, Loss: 1.3572


In [ ]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in sentences_ne
]


In [ ]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.26308657526261825,
 'precisions': [0.6080493619408217,
  0.42540355454100764,
  0.3127555988315482,
  0.23641632455928518],
 'brevity_penalty': 0.7074456183810502,
 'length_ratio': 0.7428898843629544,
 'translation_length': 7131,
 'reference_length': 9599}

In [ ]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.8664426803588867,
  0.7905800342559814,
  0.7682116031646729,
  0.7883107662200928,
  0.5715967416763306,
  0.8572221994400024,
  0.6497834920883179,
  0.8551269173622131,
  0.8138712644577026,
  0.8690667152404785,
  0.6494177579879761,
  0.8722113370895386,
  0.4686126708984375,
  0.7251790761947632,
  0.6637405157089233,
  0.7458484768867493,
  0.837470293045044,
  0.9325549602508545,
  0.8490806818008423,
  0.6827027797698975,
  0.8216825127601624,
  0.6641874313354492,
  0.6030565500259399,
  0.804516077041626,
  0.6872538328170776,
  0.9044262170791626,
  0.7351149320602417,
  0.7684889435768127,
  0.7030043601989746,
  0.8031405806541443,
  0.9090107679367065,
  0.9906212687492371,
  0.7377029061317444,
  0.9093289971351624,
  0.7158664464950562,
  0.5848265886306763,
  0.7206916213035583,
  0.6860220432281494,
  0.6425478458404541,
  0.5533080101013184,
  0.781125545501709,
  0.7633999586105347,
  0.657865047454834,
  0.8891733884811401,
  0.7909694910049438,
 

In [ ]:
model3 = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
train_transformer(model3, train_loader, optimizer, 3)

Epoch 1/3, Loss: 3.6362
Epoch 2/3, Loss: 3.6488
Epoch 3/3, Loss: 3.6330


In [ ]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in sentences_ne
]


In [ ]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.054094793492812446,
 'precisions': [0.25254266505774864,
  0.1281786231135001,
  0.07749550936617912,
  0.04683288409703504],
 'brevity_penalty': 0.5195902782836934,
 'length_ratio': 0.6043337847692468,
 'translation_length': 5801,
 'reference_length': 9599}

In [ ]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

{'precision': [0.6398988962173462,
  0.45298680663108826,
  0.3977586030960083,
  0.4507792294025421,
  0.40933656692504883,
  0.6954677104949951,
  0.5184924602508545,
  0.4689314365386963,
  0.6950495839118958,
  0.8523643612861633,
  0.4071613550186157,
  0.45782047510147095,
  0.47213783860206604,
  0.6108989119529724,
  0.3795485496520996,
  0.6061670780181885,
  0.5104516744613647,
  0.582760214805603,
  0.5577335953712463,
  0.751948356628418,
  0.45745664834976196,
  0.3526149392127991,
  0.48038560152053833,
  0.5279377698898315,
  0.6270197629928589,
  0.3664058446884155,
  0.0,
  0.525271475315094,
  0.41343238949775696,
  0.5096299052238464,
  0.5188114047050476,
  0.34831148386001587,
  0.7377027869224548,
  0.7723147869110107,
  0.549582302570343,
  0.40577346086502075,
  0.4543711543083191,
  0.3733256459236145,
  0.5991231799125671,
  0.3717361092567444,
  0.4479840099811554,
  0.7633999586105347,
  0.39981868863105774,
  0.6504856944084167,
  0.668059229850769,
  0.812

# TASK 2



In [ ]:
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
instruction = 'Convert the negative-sentiment sentence into a positive-sentiment sentence: '

In [ ]:
sentences_ne_with_ins = [f'{instruction}{s}' for s in sentences_ne]

In [ ]:
sentences_ne_with_ins

["Convert the negative-sentiment sentence into a positive-sentiment sentence: ever since joes has changed hands it's just gotten worse and worse.",
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: there is definitely not enough room in that part of the venue.',
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: so basically tasted watered down.',
 "Convert the negative-sentiment sentence into a positive-sentiment sentence: she said she'd be back and disappeared for a few minutes.",
 "Convert the negative-sentiment sentence into a positive-sentiment sentence: i can't believe how inconsiderate this pharmacy is.",
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: just left and took it off the bill.',
 "Convert the negative-sentiment sentence into a positive-sentiment sentence: it isn't terrible, but it isn't very good either.",
 'Convert the negative-sentiment sentence into a positive-sentiment sentence: d

In [ ]:
train_set = create_transformers_train_data(sentences_ne_with_ins, sentences_pos, tokenizer)
train_set

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 998
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [ ]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [ ]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 5.4096
Epoch 2/5, Loss: 4.1130
Epoch 3/5, Loss: 3.8523
Epoch 4/5, Loss: 3.6891
Epoch 5/5, Loss: 3.5804


In [ ]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in sentences_ne
]


In [ ]:
predicted_sentences

["i'm so happy with the food",
 'i would recommend this place to anyone who',
 'the food was great and the food was great',
 "she said she'd be back and",
 'i love the sand',
 'the food was great and the food was great',
 "it's a great place to go",
 'i was so happy with the food and',
 "i'm so happy with the service",
 'i love the food and the food.',
 'i love the food and the food.',
 'the food was great and the food was great',
 "i'm so happy with the food",
 "i'm so happy with the food",
 'the food was great and the food was great',
 'the food is great and the food is great',
 'the food was great and the food was great',
 "i'm so happy with the service",
 'the food was great and the food was great',
 'the food was great and the food was great',
 'i was so happy with the food.',
 "i'm so happy to be back",
 'the food is great and the food is great',
 'the food was great and the food was great',
 'i was so happy with the food and',
 'i was so happy with the food and',
 'i love the sa

In [ ]:
bleu = load('bleu')
bertscore = load('bertscore')

In [ ]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.018441853877350527,
 'precisions': [0.1939255305629686,
  0.03318294280308543,
  0.010556785288608888,
  0.0041025641025641026],
 'brevity_penalty': 0.8026391491109088,
 'length_ratio': 0.8197728930096885,
 'translation_length': 7869,
 'reference_length': 9599}

In [ ]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

{'precision': [0.5854562520980835,
  0.554582953453064,
  0.3363591432571411,
  0.796621561050415,
  0.5386558175086975,
  0.3194602131843567,
  0.6813738346099854,
  0.633374035358429,
  0.588433563709259,
  0.4344922602176666,
  0.511980414390564,
  0.361198365688324,
  0.5021499395370483,
  0.6163229942321777,
  0.4186672270298004,
  0.4030153751373291,
  0.48862388730049133,
  0.5606008768081665,
  0.45312830805778503,
  0.35429707169532776,
  0.7251588106155396,
  0.6744112968444824,
  0.4905223846435547,
  0.370357871055603,
  0.5603227615356445,
  0.6548883318901062,
  0.6149865984916687,
  0.3152862787246704,
  0.4617530405521393,
  0.48145192861557007,
  0.42413100600242615,
  0.45426875352859497,
  0.6623934507369995,
  0.6743366122245789,
  0.5222833156585693,
  0.6067133545875549,
  0.41538459062576294,
  0.6295943856239319,
  0.36813217401504517,
  0.632156491279602,
  0.470095694065094,
  0.6409972906112671,
  0.45128440856933594,
  0.5636484622955322,
  0.635641813278198

In [ ]:
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
optimizer2 = AdamW(model.parameters(), lr=0.005)

In [ ]:
train_transformer(model2, train_loader, optimizer2, 5)

Epoch 1/5, Loss: 7.0176
Epoch 2/5, Loss: 7.0439
Epoch 3/5, Loss: 7.0582
Epoch 4/5, Loss: 7.0555
Epoch 5/5, Loss: 7.0738


In [ ]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in sentences_ne
]


In [ ]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.018441853877350527,
 'precisions': [0.1939255305629686,
  0.03318294280308543,
  0.010556785288608888,
  0.0041025641025641026],
 'brevity_penalty': 0.8026391491109088,
 'length_ratio': 0.8197728930096885,
 'translation_length': 7869,
 'reference_length': 9599}

In [ ]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.5854562520980835,
  0.554582953453064,
  0.3363591432571411,
  0.796621561050415,
  0.5386558175086975,
  0.3194602131843567,
  0.6813738346099854,
  0.633374035358429,
  0.588433563709259,
  0.4344922602176666,
  0.511980414390564,
  0.361198365688324,
  0.5021499395370483,
  0.6163229942321777,
  0.4186672270298004,
  0.4030153751373291,
  0.48862388730049133,
  0.5606008768081665,
  0.45312830805778503,
  0.35429707169532776,
  0.7251588106155396,
  0.6744112968444824,
  0.4905223846435547,
  0.370357871055603,
  0.5603227615356445,
  0.6548883318901062,
  0.6149865984916687,
  0.3152862787246704,
  0.4617530405521393,
  0.48145192861557007,
  0.42413100600242615,
  0.45426875352859497,
  0.6623934507369995,
  0.6743366122245789,
  0.5222833156585693,
  0.6067133545875549,
  0.41538459062576294,
  0.6295943856239319,
  0.36813217401504517,
  0.632156491279602,
  0.470095694065094,
  0.6409972906112671,
  0.45128440856933594,
  0.5636484622955322,
  0.635641813278198

In [ ]:
model3 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
train_transformer(model3, train_loader, optimizer, 10)

Epoch 1/10, Loss: 7.0721
Epoch 2/10, Loss: 7.0379
Epoch 3/10, Loss: 7.0432
Epoch 4/10, Loss: 7.0403
Epoch 5/10, Loss: 7.0380
Epoch 6/10, Loss: 7.0626
Epoch 7/10, Loss: 7.0271
Epoch 8/10, Loss: 7.0180
Epoch 9/10, Loss: 7.0524
Epoch 10/10, Loss: 7.0432


In [ ]:
predicted_sentences = [
    decode_with_transformer(s, tokenizer, model)
    for s in sentences_ne
]


In [ ]:
bleu.compute(predictions=predicted_sentences, references=reference_sentences)

{'bleu': 0.018441853877350527,
 'precisions': [0.1939255305629686,
  0.03318294280308543,
  0.010556785288608888,
  0.0041025641025641026],
 'brevity_penalty': 0.8026391491109088,
 'length_ratio': 0.8197728930096885,
 'translation_length': 7869,
 'reference_length': 9599}

In [ ]:
bertscore.compute(predictions=predicted_sentences, references=reference_sentences, model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.5854562520980835,
  0.554582953453064,
  0.3363591432571411,
  0.796621561050415,
  0.5386558175086975,
  0.3194602131843567,
  0.6813738346099854,
  0.633374035358429,
  0.588433563709259,
  0.4344922602176666,
  0.511980414390564,
  0.361198365688324,
  0.5021499395370483,
  0.6163229942321777,
  0.4186672270298004,
  0.4030153751373291,
  0.48862388730049133,
  0.5606008768081665,
  0.45312830805778503,
  0.35429707169532776,
  0.7251588106155396,
  0.6744112968444824,
  0.4905223846435547,
  0.370357871055603,
  0.5603227615356445,
  0.6548883318901062,
  0.6149865984916687,
  0.3152862787246704,
  0.4617530405521393,
  0.48145192861557007,
  0.42413100600242615,
  0.45426875352859497,
  0.6623934507369995,
  0.6743366122245789,
  0.5222833156585693,
  0.6067133545875549,
  0.41538459062576294,
  0.6295943856239319,
  0.36813217401504517,
  0.632156491279602,
  0.470095694065094,
  0.6409972906112671,
  0.45128440856933594,
  0.5636484622955322,
  0.635641813278198